## Global statistics of tracked eddies 

In [ ]:
import os 
import shutil
import numpy as np
import cartopy 
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from datetime import datetime, timedelta
import xarray as xr
import matplotlib.pyplot as plt
from cartopy.mpl.ticker import (LongitudeFormatter, LatitudeFormatter,LatitudeLocator)
import scipy
import matplotlib as mpl


### define directories where track data is stored

In [ ]:
# track directory for wavelength = 700 km and shape error = 70%
trackdir_wv700_se70='/path/to/track/data/erc1011_eddytrack/wv_'+str(700)+'_se_'+str(70)+'/tracks/'
# track directory for wavelength = 700 km and shape error = 30%
trackdir_wv700_se30='/path/to/track/data/erc1011_eddytrack/wv_'+str(700)+'_se_'+str(30)+'/tracks/'
# track directory for wavelength = 200 km and shape error = 70%
trackdir_wv200_se70='/path/to/track/data/erc1011_eddytrack/wv_'+str(200)+'_se_'+str(70)+'/tracks/'
# track directory for wavelength = 200 km and shape error = 30%
trackdir_wv200_se30='/path/to/track/data/erc1011_eddytrack/wv_'+str(200)+'_se_'+str(30)+'/tracks/'


#open datasets for cyclonic and anticyclonic tracks 
dstracks_wv700_se70_c = xr.open_dataset(trackdir_wv700_se70+'cyclonic'+'_tracks.nc')
dstracks_wv700_se70_ac = xr.open_dataset(trackdir_wv700_se70+'anticyclonic'+'_tracks.nc')
dstracks_wv700_se30_c = xr.open_dataset(trackdir_wv700_se30+'cyclonic'+'_tracks.nc')
dstracks_wv700_se30_ac = xr.open_dataset(trackdir_wv700_se30+'anticyclonic'+'_tracks.nc')
dstracks_wv200_se70_c = xr.open_dataset(trackdir_wv200_se70+'cyclonic'+'_tracks.nc')
dstracks_wv200_se70_ac = xr.open_dataset(trackdir_wv200_se70+'anticyclonic'+'_tracks.nc')
dstracks_wv200_se30_c = xr.open_dataset(trackdir_wv200_se30+'cyclonic'+'_tracks.nc')
dstracks_wv200_se30_ac = xr.open_dataset(trackdir_wv200_se30+'anticyclonic'+'_tracks.nc')

In [ ]:
# look at the data
display(dstracks_wv700_se70_ac)

### plot eddy tracks from one of the data sets

In [ ]:
%matplotlib inline
lonshift = 0
proj=ccrs.PlateCarree(central_longitude = lonshift)
fig, ax = plt.subplots(1, 1, figsize=(80, 30), subplot_kw={'projection': proj})
ax.set_title('Anticyclonic ocean-eddy tracks', loc='center', fontsize=50)
ax.set_title('2002-01-01 to 2008-12-31', loc='left', fontsize=50)
ax.set_title('(wavelength,shape_error)'+str(int(wavelength))+' '+str(int(shape_error)), loc='right', fontsize=50)
gl = ax.gridlines(draw_labels=True, y_inline=True) # need cartopy version 0.18!!!
gl.top_labels = False
gl.xformatter = LongitudeFormatter()
gl.yformatter = LatitudeFormatter()

land = cfeature.NaturalEarthFeature(
        category='physical',
        name='land',
        scale='10m',
        facecolor='grey')
ax.add_feature(land)

ax.scatter(dstracks_wv700_se70_ac.longitude.values, dstracks_wv700_se70_ac.latitude.values, marker='o', alpha=0.01, color='red')

### Define function to average eddy radius and speed along track and then bin according to latitude

In [ ]:
def eddy_lonav(dstracks):
    uniq_vals, uniq_idx = np.unique(dstracks.track.values, return_index=True)
    #print(uniq_idx)
    eddy_idx = np.split(dstracks.track.values,uniq_idx)
    eddy_uavg = np.split(dstracks.uavg_profile.values,uniq_idx)
    eddy_radius = np.split(dstracks.effective_radius.values, uniq_idx)
    eddy_latitude = np.split(dstracks.latitude.values, uniq_idx)

    eddy_idx_av = np.empty(len(eddy_idx))
    eddy_uavg_av = np.empty(len(eddy_idx))
    eddy_radius_av = np.empty(len(eddy_idx))
    eddy_latitude_av = np.empty(len(eddy_idx))
    for i in range(len(eddy_idx)):
        eddy_idx_av[i] = np.mean(eddy_idx[i])
        eddy_uavg_av[i] = np.mean(eddy_uavg[i])
        eddy_radius_av[i] = np.mean(eddy_radius[i])
        eddy_latitude_av[i] = np.mean(eddy_latitude[i])
    #remove nan from first value
    eddy_idx_av = eddy_idx_av[1:]
    eddy_uavg_av = eddy_uavg_av[1:]
    eddy_radius_av = eddy_radius_av[1:]
    eddy_latitude_av = eddy_latitude_av[1:]

    latbins = np.arange(-90,100,10)
    ibin = np.digitize(eddy_latitude_av, latbins,right=False)
    #latbins[ibin[0]],eddy_latitude_av[0]
    midlatbins = np.arange(-85,95,10)


    eddy_uavg_lonav = np.empty(18)
    eddy_radius_lonav = np.empty(18)
    for x in range(len(latbins)-1):
        #print(x+1)
        eddy_uavg_lonav[x-1] = np.mean(eddy_uavg_av[(ibin == int(x))])
        eddy_radius_lonav[x-1] = np.mean(eddy_radius_av[(ibin == int(x))])
    return eddy_uavg_lonav, eddy_radius_lonav

### plot zonally averaged eddy radius and eddy speed computed with 4 different parameter combinations

In [ ]:
eddy_uavg_lonav, eddy_radius_lonav = eddy_lonav(dstracks_ac)
fig, ax = plt.subplots(2, 2, figsize=(15, 8))
ax=np.ndarray.flatten(ax)
paramstr=['wv700_se70','wv700_se30','wv200_se70','wv200_se30']
colors = ['blue','red','blue','red']
linestyles = ['-','-','--','--']
fig.suptitle('Global eddy statistics from 2002-01-01 to 2008-12-31 averaged over track and zonally binned')
for i in range(4):
    exec("eddy_uavg_lonav_ac, eddy_radius_lonav_ac = eddy_lonav(dstracks_%s_ac)" % (paramstr[i]))
    exec("eddy_uavg_lonav_c, eddy_radius_lonav_c = eddy_lonav(dstracks_%s_c)" % (paramstr[i]))
    ax[0].plot(midlatbins,eddy_uavg_lonav_ac, color=colors[i], label=paramstr[i], ls=linestyles[i])
    ax[1].plot(midlatbins,eddy_uavg_lonav_c, color=colors[i], label=paramstr[i], ls=linestyles[i])
    ax[0].set_xlabel('latitude')
    ax[0].set_ylabel('average ac eddy speed (m/s)')
    ax[1].set_xlabel('latitude')
    ax[1].set_ylabel('average c eddy speed (m/s)')
    ax[2].plot(midlatbins,eddy_radius_lonav_ac/1000., color=colors[i], label=paramstr[i], ls=linestyles[i])
    ax[3].plot(midlatbins,eddy_radius_lonav_c/1000., color=colors[i], label=paramstr[i], ls=linestyles[i])
    ax[2].set_xlabel('latitude')
    ax[2].set_ylabel('average ac eddy radius (km)')
    ax[3].set_xlabel('latitude')
    ax[3].set_ylabel('average c eddy radius (km)')
    ax[0].legend()
    ax[1].legend()
    ax[2].legend()
    ax[3].legend()
fig.savefig('/path_to_plot/global_eddy_stats.png')